# Capstone Project  week 4- The Battle of the Neighborhoods (Week 2)

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a coffee shop. Specifically, this report will be targeted to stakeholders interested in opening an **Coffee shop** in **Nha Trang city**, Vietnam.

Since there are lots of coffee shops in Nha Trang we will try to detect **locations that are not already crowded with coffee shops**. Because our client already had a successful coffee shop, we are also particularly interested in **areas that are similar with our client's coffee shop**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing coffee shops in the neighborhood (any type of coffee shop)
* similar venues between the location of the client's coffee shop and new potential locations
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of coffee shops and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Nha Trang city center will be obtained using **Google Maps API geocoding** of well known Nha Trang location (Thap Tram Huong)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Nha Trang city center.

Let's first find the latitude & longitude of Nha Trang city center, using specific, well known address and Google Maps geocoding API.

In [58]:
# The code was removed by Watson Studio for sharing.
!conda install -y -c conda-forge geocoder
import geocoder # import geocoder


Solving environment: done

# All requested packages already installed.



In [248]:
# Google API key
google_api_key = '<removed due to sensitive information>'

# Foursquare API key/secret
foursquare_client_id = '<removed due to sensitive information>'
foursquare_client_secret = '<removed due to sensitive information>'
VERSION = '20180605' # Foursquare API version

In [59]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Thap Tram Huong, Nha Trang, Vietnam'
nhatrang_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, nhatrang_center))

Coordinate of Thap Tram Huong, Nha Trang, Vietnam: [12.2406003, 109.1969227]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Thap Tram Huong. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [60]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Nha Trang center longitude={}, latitude={}'.format(nhatrang_center[1], nhatrang_center[0]))
x, y = lonlat_to_xy(nhatrang_center[1], nhatrang_center[0])
print('Nha Trang center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Nha Trang center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Nha Trang center longitude=109.1969227, latitude=12.2406003
Nha Trang center UTM X=14235309.128797397, Y=11966744.659310125
Nha Trang center longitude=109.19691887988347, latitude=12.240602661512039


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [64]:
# City center in Cartesian coordinates
nhatrang_center_x, nhatrang_center_y = lonlat_to_xy(nhatrang_center[1], nhatrang_center[0]) 

# Vertical offset for hexagonal grid cells
k = math.sqrt(3) / 2 
x_min = nhatrang_center_x - 6000
x_step = 600
y_min = nhatrang_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(nhatrang_center_x, nhatrang_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [65]:
#!pip install folium

import folium

In [71]:
map_nhatrang = folium.Map(location=nhatrang_center, zoom_start=15)
folium.Marker(nhatrang_center, popup='Thap Tram Huong').add_to(map_nhatrang)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=70, color='blue', fill=False).add_to(map_nhatrang)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_nhatrang

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Thap Tram Huong. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [72]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, nhatrang_center[0], nhatrang_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(nhatrang_center[0], nhatrang_center[1], addr))

Reverse geocoding check
-----------------------
Address of [12.2406003, 109.1969227] is: Đường Trần Phú, Lộc Thọ, Thành phố Nha Trang, Khánh Hòa 650000, Vietnam


In [73]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [84]:
# We don't need country part of address
for i,a in enumerate(addresses):
    addresses[i] = addresses[i].replace(', Thành phố Nha Trang', '')
    addresses[i] = addresses[i].replace(', Thành Phố Nha Trang', '')
    addresses[i] = addresses[i].replace(', Nha Trang City', '')
    addresses[i] = addresses[i].replace(', Tp. Nha Trang', '')
    addresses[i] = addresses[i].replace(', Khánh Hòa 650000', '')
    addresses[i] = addresses[i].replace(', Khanh Hoa Province', '')
    addresses[i] = addresses[i].replace(', Khanh Hoa', '')
    addresses[i] = addresses[i].replace(', Khánh Hòa', '')
    addresses[i] = addresses[i].replace(', Tỉnh Khánh Hòa', '')
    addresses[i] = addresses[i].replace(', Vietnam', '')
addresses

['23 Thai Nguyen Street, Phuoc Tan Ward, Phước Tân',
 '04 Đường Đề Pô, Phước Tân',
 '17/7, Đường số 1, Máy Nước, Phường Phước Tân, Phước Tân',
 '01 Chi Lăng, Phước Tân',
 '11a Trần Khánh Dư, Phước Tân',
 '21a Lê Chân, Phước Tân 651751',
 '12 Thi Sách, Phước Hoà',
 '67A Yersin, Vạn Thắng',
 '62 Thái Nguyên, Phương Sài',
 '29c Thái Nguyên, Phước Tân',
 '36 Đường Đề Pô, Phước Tân',
 '100 Nguyễn Trãi, Phước Tân',
 '7 Lạc Long Quân, Phước Tân',
 '139 Nguyễn Trãi, Phước Tiến',
 '16 Phù Đổng, Phước Tiến',
 '20 Trần Nhật Duật, Phước Tiến',
 '9a Cửu Long, Phước Hoà',
 '22 Yết Kiêu, Vạn Thắng',
 '50, Bis Yersin, Vạn Thắng',
 '53 Lê Thành Phương, Vạn Thắng',
 '1a Thái Nguyên, Phước Tân',
 '10 Nguyen Trai Street, Phuoc Tan Ward, Phước Tân',
 '60 Nguyễn Trãi, Phước Tân',
 '28A Hoa Lư, Phước Tiến',
 '43-41 Mạc Đĩnh Chi, Phước Tiến',
 '9b Mê Linh, Phước Tiến',
 '54 Phù Đổng, Phước Tiến',
 '240b Ngô Gia Tự, Phước Hoà',
 '50 Trần Nhật Duật, Phước Hoà',
 '10 Lam Sơn, Phước Hoà',
 '6/1 Lý Thánh Tôn, Vạn 

Looking good. Let's now place all this into a Pandas dataframe.

In [85]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"23 Thai Nguyen Street, Phuoc Tan Ward, Phước Tân",12.247832,109.186305,1.423351e+07,1.196103e+07,5992.495307
1,"04 Đường Đề Pô, Phước Tân",12.246617,109.185922,1.423411e+07,1.196103e+07,5840.376700
2,"17/7, Đường số 1, Máy Nước, Phường Phước Tân, ...",12.245403,109.185540,1.423471e+07,1.196103e+07,5747.173218
3,"01 Chi Lăng, Phước Tân",12.244188,109.185158,1.423531e+07,1.196103e+07,5715.767665
4,"11a Trần Khánh Dư, Phước Tân",12.242974,109.184776,1.423591e+07,1.196103e+07,5747.173218
5,"21a Lê Chân, Phước Tân 651751",12.241759,109.184394,1.423651e+07,1.196103e+07,5840.376700
6,"12 Thi Sách, Phước Hoà",12.240545,109.184012,1.423711e+07,1.196103e+07,5992.495307
7,"67A Yersin, Vạn Thắng",12.249328,109.187948,1.423261e+07,1.196155e+07,5855.766389
8,"62 Thái Nguyên, Phương Sài",12.248114,109.187566,1.423321e+07,1.196155e+07,5604.462508
9,"29c Thái Nguyên, Phước Tân",12.246899,109.187183,1.423381e+07,1.196155e+07,5408.326913


...and let's now save/persist this data into local file.

In [86]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on coffee shops in each neighborhood.

We're interested in venues in 'coffee' category, but only those that are proper coffee shop - restaurant, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'coffee' in category name.

Foursquare credentials are defined in hidden cell bellow.

In [88]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

coffee_category = '4bf58dd8d48988d1e0931735' # 'Root' category for all coffee shop venues

def is_coffee_shop(categories, specific_filter=None):
    coffee_words = ['coffee', 'ca phe']
    coffee = False
    specific = True # always specific
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in coffee_words:
            if r in category_name:
                coffee = True
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            coffee = True
    return coffee, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [90]:
# Let's now go over our neighborhood locations and get nearby coffee shop
# we'll also maintain a dictionary of all found coffee shop

import pickle

def get_coffee_shops(lats, lons):
    coffee_shops = {}
    location_coffee_shops = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage 
        # so we don't miss any place 
        # (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, coffee_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_coffee_shops = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_cs, _ = is_coffee_shop(venue_categories, specific_filter=[])
            if is_cs:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                coffee_shop = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, True, x, y)
                if venue_distance<=300:
                    area_coffee_shops.append(coffee_shop)
                coffee_shops[venue_id] = coffee_shop
        location_coffee_shops.append(area_coffee_shops)
        print(' .', end='')
    print(' done.')
    return coffee_shops, [], location_coffee_shops

# Try to load from local file system in case we did this before
coffee_shops = {}
location_coffee_shops = []
loaded = False
try:
    with open('coffee_shops_350.pkl', 'rb') as f:
        coffee_shops = pickle.load(f)
    with open('location_coffee_shops_350.pkl', 'rb') as f:
        location_coffee_shops = pickle.load(f)
    print('Coffee shop data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    coffee_shops, _, location_coffee_shops = get_coffee_shops(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('coffee_shops_350.pkl', 'wb') as f:
        pickle.dump(coffee_shops, f)
    with open('location_coffee_shops_350.pkl', 'wb') as f:
        pickle.dump(location_coffee_shops, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [91]:
import numpy as np

print('Total number of coffee shops:', len(coffee_shops))
print('Average number of coffee shops in neighborhood:', np.array([len(r) for r in location_coffee_shops]).mean())

Total number of coffee shops: 55
Average number of coffee shops in neighborhood: 1.4038461538461537


In [92]:
print('List of all coffee_shops')
print('-----------------------')
for r in list(coffee_shops.values())[:10]:
    print(r)
print('...')
print('Total:', len(coffee_shops))

List of all coffee_shops
-----------------------
('51c3a4b7498eaaafb1ee5043', 'Thời Gian', 12.249537261935563, 109.18616108813511, '64 Yersin, Nha Trang, Tỉnh Khánh Hòa, Việt Nam', 314, True, 14232759.598903043, 11960725.325768027)
('51e0a002498e0251c17f8bed', 'Hoàng Tuấn', 12.247032714258417, 109.18908190103045, '01 Nguyen Trai, Nha Trang, Tỉnh Khánh Hòa, Việt Nam', 304, True, 14233487.60101751, 11962369.57758499)
('5067be02e4b01301ec6c69f8', 'cà phê dendro', 12.250051498413086, 109.18656921386719, 'Việt Nam', 248, True, 14232471.848083545, 11960834.514390402)
('57737fc4498e6476d7f29876', 'kaffetino', 12.249428, 109.187936, 'Yersin 46, Việt Nam', 293, True, 14232565.325257173, 11961527.332142923)
('5688e97c498e55b3ada63fa4', 'Parista The Coffee', 12.249543, 109.1878, '46 Yersin, Việt Nam', 301, True, 14232532.159192191, 11961451.0044285)
('4f1a508fe4b0288a01e93a52', 'Phú Sĩ Coffee', 12.249560009446153, 109.18780440599092, 'Việt Nam', 303, True, 14232523.888773892, 11961450.584268965)


In [93]:
print('Coffee Shops around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_coffee_shops[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Coffee Shop around location {}: {}'.format(i+1, names))

Coffee Shops around location
---------------------------
Coffee Shop around location 101: Coffee Museum, Cool Coffee - NT center, Milano coffee, Me Trang coffee, Yasaka Coffee
Coffee Shop around location 102: Me Trang coffee, Milano coffee
Coffee Shop around location 103: Me Trang coffee, Vuon Dua, Milano coffee, Iced Coffee Vincom Center, Terrace Cafe
Coffee Shop around location 104: Me Trang coffee, Vuon Dua, Iced Coffee Vincom Center, Milano coffee
Coffee Shop around location 105: Windmills coffee, Young Startup Dessert & Cafe, Vuon Dua, Iced Coffee Vincom Center, Me Trang coffee, New Let It Be Coffee Shop Bach Dang
Coffee Shop around location 106: Windmills coffee, Young Startup Dessert & Cafe, Vuon Dua, Iced Coffee Vincom Center, New Let It Be Coffee Shop Bach Dang
Coffee Shop around location 107: Venice Coffee, Young Startup Dessert & Cafe, Tasty!, New Let It Be Coffee Shop Bach Dang
Coffee Shop around location 108: Venice Coffee, Iced Coffee, Young Startup Dessert & Cafe, Tasty!

Let's now see all the collected coffee shops in our area of interest on map

In [94]:
map_nhatrang = folium.Map(location=nhatrang_center, zoom_start=15)
folium.Marker(nhatrang_center, popup='Thap Tram Huong').add_to(map_nhatrang)
for res in coffee_shops.values():
    lat = res[2]; lon = res[3]
    color = 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_nhatrang)
map_nhatrang

Looking good. So now we have all the coffee shops in area within few kilometers from Thap Tram Huong! We also know which coffee shops exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new coffee shop !

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Nha Trang that have low coffee shop density. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every coffee shop within 6km from Nha Trang center** (Thap Tram Huong). We have also **identified coffee shop** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**coffee shop density**' across different areas of Nha Trang - we will use **heatmaps** to identify a few promising areas close to center with low number of coffee shops in general and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two coffee shops in radius of 250 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of coffee shops in every area candidate**:

In [95]:
location_coffee_shops_count = [len(res) for res in location_coffee_shops]

df_locations['Coffee shop in area'] = location_coffee_shops_count

print('Average number of coffee shops in every area with radius=300m:', np.array(location_coffee_shops_count).mean())

df_locations.head(10)

Average number of coffee shops in every area with radius=300m: 1.4038461538461537


,Address,Latitude,Longitude,X,Y,Distance from center,Coffee shop in area
0,"23 Thai Nguyen Street, Phuoc Tan Ward, Phước Tân",12.247832,109.186305,1.423351e+07,1.196103e+07,5992.495307,6
1,"04 Đường Đề Pô, Phước Tân",12.246617,109.185922,1.423411e+07,1.196103e+07,5840.376700,1
2,"17/7, Đường số 1, Máy Nước, Phường Phước Tân, ...",12.245403,109.185540,1.423471e+07,1.196103e+07,5747.173218,0
3,"01 Chi Lăng, Phước Tân",12.244188,109.185158,1.423531e+07,1.196103e+07,5715.767665,0
4,"11a Trần Khánh Dư, Phước Tân",12.242974,109.184776,1.423591e+07,1.196103e+07,5747.173218,2
5,"21a Lê Chân, Phước Tân 651751",12.241759,109.184394,1.423651e+07,1.196103e+07,5840.376700,3
6,"12 Thi Sách, Phước Hoà",12.240545,109.184012,1.423711e+07,1.196103e+07,5992.495307,3
7,"67A Yersin, Vạn Thắng",12.249328,109.187948,1.423261e+07,1.196155e+07,5855.766389,1
8,"62 Thái Nguyên, Phương Sài",12.248114,109.187566,1.423321e+07,1.196155e+07,5604.462508,2
9,"29c Thái Nguyên, Phước Tân",12.246899,109.187183,1.423381e+07,1.196155e+07,5408.326913,3


OK, now let's calculate the **distance to nearest coffee shop from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [96]:
distances_to_coffee_shop = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in coffee_shops.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_coffee_shop.append(min_distance)

df_locations['Distance to coffee shop'] = distances_to_coffee_shop

In [160]:
df_locations.describe()

,Latitude,Longitude,X,Y,Distance from center,Coffee shop in area,Distance to coffee shop
count,364.000000,364.000000,3.640000e+02,3.640000e+02,364.000000,364.000000,364.000000
mean,12.240603,109.196919,1.423531e+07,1.196674e+07,4007.610159,1.403846,1564.671730
std,0.006325,0.006537,2.979766e+03,3.039900e+03,1419.442075,2.214176,1307.507689
min,12.228049,109.184012,1.422961e+07,1.196103e+07,300.000000,0.000000,34.300184
25%,12.235513,109.191658,1.423291e+07,1.196415e+07,2954.657341,0.000000,580.665595
50%,12.240602,109.196918,1.423531e+07,1.196674e+07,4242.627429,0.000000,1084.498995
75%,12.245687,109.202181,1.423771e+07,1.196934e+07,5204.805472,2.000000,2277.832232
max,12.253166,109.209829,1.424101e+07,1.197246e+07,5992.495307,11.000000,5530.547741


In [97]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Coffee shop in area,Distance to coffee shop
0,"23 Thai Nguyen Street, Phuoc Tan Ward, Phước Tân",12.247832,109.186305,1.423351e+07,1.196103e+07,5992.495307,6,582.035941
1,"04 Đường Đề Pô, Phước Tân",12.246617,109.185922,1.423411e+07,1.196103e+07,5840.376700,1,879.054591
2,"17/7, Đường số 1, Máy Nước, Phường Phước Tân, ...",12.245403,109.185540,1.423471e+07,1.196103e+07,5747.173218,0,1388.059117
3,"01 Chi Lăng, Phước Tân",12.244188,109.185158,1.423531e+07,1.196103e+07,5715.767665,0,1323.441885
4,"11a Trần Khánh Dư, Phước Tân",12.242974,109.184776,1.423591e+07,1.196103e+07,5747.173218,2,1003.554511
5,"21a Lê Chân, Phước Tân 651751",12.241759,109.184394,1.423651e+07,1.196103e+07,5840.376700,3,991.334904
6,"12 Thi Sách, Phước Hoà",12.240545,109.184012,1.423711e+07,1.196103e+07,5992.495307,3,695.523156
7,"67A Yersin, Vạn Thắng",12.249328,109.187948,1.423261e+07,1.196155e+07,5855.766389,1,48.653057
8,"62 Thái Nguyên, Phương Sài",12.248114,109.187566,1.423321e+07,1.196155e+07,5604.462508,2,644.151666
9,"29c Thái Nguyên, Phước Tân",12.246899,109.187183,1.423381e+07,1.196155e+07,5408.326913,3,881.780700


In [98]:
print('Average distance to closest coffee shop from each area center:', df_locations['Distance to coffee shop'].mean())

Average distance to closest coffee shop from each area center: 1564.6717303506227


OK, so **on average coffee shop can be found within ~1500m** from every area center candidate. That's fairly close, so we need to filter our areas carefully!

Let's crete a map showing **heatmap / density of coffee shops** and try to extract some meaningfull info from that. Also, let's show **borders of Nha Trang boroughs** on our map and a few circles indicating distance of 1km, 2km and 3km from Thap Tram Huong.

In [100]:
coffee_shop_latlons = [[res[2], res[3]] for res in coffee_shops.values()]

In [105]:
from folium import plugins
from folium.plugins import HeatMap

map_nhatrang = folium.Map(location=nhatrang_center, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(map_nhatrang) #cartodbpositron cartodbdark_matter
HeatMap(coffee_shop_latlons).add_to(map_nhatrang)
folium.Marker(nhatrang_center).add_to(map_nhatrang)
folium.Circle(nhatrang_center, radius=500, fill=False, color='white').add_to(map_nhatrang)
folium.Circle(nhatrang_center, radius=1000, fill=False, color='white').add_to(map_nhatrang)
folium.Circle(nhatrang_center, radius=1500, fill=False, color='white').add_to(map_nhatrang)
map_nhatrang

Looks like a few pockets of low coffee shop density closest to city center can be found **north-west (~750m) and south-west (~1km) from Thap Tram Huong**. 

This map is not so 'hot' but it also indicates higher density of existing coffee shop directly north and south from Thap Tram Huong, with closest pockets of **low coffee shop density positioned south-west and north-west from city center**.

By domain knowledge, we know that the **north-west** areas are very expensive locations. So that we focus our analysis on areas *south-west from Nha Trang center* - we will move the center of our area of interest and reduce it's size to have a radius of **2.5km**. This places our location candidates mostly in boroughs between **Phuoc Hoa** and **Tan Lap**.

Let's also create new, more dense grid of location candidates restricted to our new region of interest. We create a new "roi_center" and shift it from Nha Trang center to (+2500, -2500) to make it between **Phuoc Hoa** and **Tan Lap**. We also create a new marker with another icon for this location

In [135]:
roi_x_min = nhatrang_center_x
roi_y_max = nhatrang_center_y
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min + 2500
roi_center_y = roi_y_max - 2500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_nhatrang = folium.Map(location=roi_center, zoom_start=15)
HeatMap(coffee_shop_latlons).add_to(map_nhatrang)
folium.Marker(nhatrang_center).add_to(map_nhatrang)
folium.Marker(roi_center, icon=folium.Icon(color='cloud')).add_to(map_nhatrang)
folium.Circle(roi_center, radius=1000, color='white', fill=True, fill_opacity=0.4).add_to(map_nhatrang)
map_nhatrang

In [136]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

2261 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of coffee shops in vicinity** (we'll use radius of **500 meters**) and **distance to closest coffee shop**.

In [139]:
def count_coffee_shops_nearby(x, y, coffee_shops, radius=500):    
    count = 0
    for res in coffee_shops.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_coffee_shop(x, y, coffee_shops):
    d_min = 100000
    for res in coffee_shops.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_coffee_shop_counts = []
roi_coffee_shop_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_coffee_shops_nearby(x, y, coffee_shops, radius=250)
    roi_coffee_shop_counts.append(count)
    distance = find_nearest_coffee_shop(x, y, coffee_shops)
    roi_coffee_shop_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [140]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Coffee shop nearby':roi_coffee_shop_counts,
                                 'Distance to closest coffee shop':roi_coffee_shop_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Coffee shop nearby,Distance to closest coffee shop
0,12.238782,109.185070,1.423776e+07,1.196174e+07,1,213.564352
1,12.238579,109.185007,1.423786e+07,1.196174e+07,1,242.312564
2,12.239840,109.185599,1.423721e+07,1.196183e+07,0,378.661333
3,12.239638,109.185535,1.423731e+07,1.196183e+07,0,305.970442
4,12.239436,109.185472,1.423741e+07,1.196183e+07,0,252.688381
5,12.239233,109.185408,1.423751e+07,1.196183e+07,1,232.561656
6,12.239031,109.185344,1.423761e+07,1.196183e+07,1,184.550104
7,12.238829,109.185280,1.423771e+07,1.196183e+07,1,131.012927
8,12.238626,109.185217,1.423781e+07,1.196183e+07,1,142.372867
9,12.238424,109.185153,1.423791e+07,1.196183e+07,1,208.268287


Looking good. What we have now is a clear indication of zones with low number of coffee shop in vicinity.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [142]:
good_coffee_shop_count = np.array((df_roi_locations['Coffee shop nearby']<=2))
print('Locations with no more than two coffee shop nearby:', good_coffee_shop_count.sum())

good_coffee_shop_distance = np.array(df_roi_locations['Distance to closest coffee shop']>=400)
print('Locations with no coffee shop within 400m:', good_coffee_shop_distance.sum())

good_locations = np.logical_and(good_coffee_shop_count, good_coffee_shop_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two coffee shop nearby: 2259
Locations with no coffee shop within 400m: 1629
Locations with both conditions met: 1629


In [153]:
from sklearn.cluster import KMeans

number_of_clusters = 10

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_nhatrang = folium.Map(location=roi_center, zoom_start=16)
folium.TileLayer('cartodbpositron').add_to(map_nhatrang)
HeatMap(coffee_shop_latlons).add_to(map_nhatrang)
folium.Circle(roi_center, radius=100, color='white', fill=True, fill_opacity=0.4).add_to(map_nhatrang)
folium.Marker(nhatrang_center).add_to(map_nhatrang)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=100, color='green', fill=True, fill_opacity=0.25).add_to(map_nhatrang) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nhatrang)
map_nhatrang

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [156]:
map_nhatrang = folium.Map(location=roi_center, zoom_start=16)
folium.Marker(nhatrang_center).add_to(map_nhatrang)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=100, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_nhatrang)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nhatrang)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=100, color='green', fill=False).add_to(map_nhatrang) 
map_nhatrang

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [157]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', Vietnam', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, nhatrang_center_x, nhatrang_center_y)
    print('{}{} => {:.1f}km from Thap Tram Huong'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000 => 4.5km from Thap Tram Huong
Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000 => 4.9km from Thap Tram Huong
75/1 Nguyễn Thị Minh Khai, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000 => 4.1km from Thap Tram Huong
58 Trần Nhật Duật, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000 => 5.5km from Thap Tram Huong
89A Bạch Đằng, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000 => 2.0km from Thap Tram Huong
Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khánh Hòa => 5.4km from Thap Tram Huong
160a Mê Linh, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000 => 3.5km from Thap Tram Huong
Biệt Thự, Thành phố Nha Trang, Khánh Hòa 650000    => 3.9km from Thap Tram Huong
34/2/11 Nguyễn Thiện Thuật, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000 => 2.8km from Thap Tram Huong
75 Hồng Bàng, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000 => 3.2km from Thap Tram Huong


This concludes our analysis. We have created 10 addresses representing centers of zones containing locations with low number of coffee shops nearby, all zones being fairly close to city center (all less than 4km from Thap Tram Huong, and about half of those less than 2km from Thap Tram Huong). Although zones are shown on map with a radius of ~100 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential coffee shop locations. Most of the zones are located in **Phuoc Hoa** and **Tan Lap** boroughs.

In [159]:
map_nhatrang = folium.Map(location=roi_center, zoom_start=16)
folium.Circle(nhatrang_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_nhatrang)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_nhatrang) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_nhatrang)
map_nhatrang

## Clustering <a name="results"></a>

Because our client had a coffee shop and it is quite success, we want to check the similarity of each of them with the success coffee shop. First, we will add the success coffee shop to the above candidate. Next, we will use FourSquare to find out the venues nearby each of location. Finally, we will cluster all (10 candidate + 1 current coffee shop) to 3 group. Then, the best match candidates are the one that grouped in same group with the success coffee shop

In [164]:
# get coordinate of current success coffee shop
success_coffee_shop_address = '40 Le Dai Hanh, Nha Trang, Viet Nam'
success_coffee_shop_coordinates = get_coordinates(google_api_key, success_coffee_shop_address)
print('Coordinate of {}: {}'.format(success_coffee_shop_address, success_coffee_shop_coordinates))

Coordinate of 40 Le Dai Hanh, Nha Trang, Viet Nam: [12.2402513, 109.1882175]


In [167]:
# add success coffee shop to above list
cluster_centers.append((success_coffee_shop_coordinates[1], success_coffee_shop_coordinates[0]))
cluster_centers

[(109.18735854453924, 12.239193561521004),
 (109.18959278651953, 12.233001804495256),
 (109.18977621736579, 12.235607269516706),
 (109.18581834024927, 12.236520375009738),
 (109.19263147760965, 12.240089203781116),
 (109.1872811268575, 12.234373192681614),
 (109.18992867330411, 12.237969172715324),
 (109.19224526089269, 12.233648289771844),
 (109.19280647371826, 12.236189806677595),
 (109.189999747699, 12.240600867063888),
 (109.1882175, 12.2402513)]

In [169]:
candidate_area_addresses.append(success_coffee_shop_address)
candidate_area_addresses

['56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000',
 'Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000',
 '75/1 Nguyễn Thị Minh Khai, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000',
 '58 Trần Nhật Duật, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000',
 '89A Bạch Đằng, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000',
 'Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khánh Hòa',
 '160a Mê Linh, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000',
 'Biệt Thự, Thành phố Nha Trang, Khánh Hòa 650000',
 '34/2/11 Nguyễn Thiện Thuật, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000',
 '75 Hồng Bàng, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000',
 '40 Le Dai Hanh, Nha Trang, Viet Nam']

In [173]:
# get nearby venues of all locations 
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    foursquare_client_id, 
    foursquare_client_secret, 
    VERSION, 
    success_coffee_shop_coordinates[0], 
    success_coffee_shop_coordinates[1], 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d92f75eb77c77002ca32fe9'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 27,
  'suggestedBounds': {'ne': {'lat': 12.244751304500005,
    'lng': 109.19281358437249},
   'sw': {'lat': 12.235751295499997, 'lng': 109.1836214156275}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5274ff3911d2ad0ff5dacf44',
       'name': 'An Cafe',
       'location': {'address': '40 Lê Đại Hành',
        'lat': 12.240179839076696,
        'lng': 109.18808996695357,
        'labeledLatLngs': [{'label': 'display',
          'lat': 12.240179839

In [175]:
# inspect the venue
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,An Cafe,Café,12.240180,109.188090
1,Kiwami,Sushi Restaurant,12.240153,109.191731
2,69 Bach Dang,Asian Restaurant,12.240993,109.190783
3,Bún Cá Nguyên Loan,Asian Restaurant,12.241141,109.188283
4,Coriander,Thai Restaurant,12.243421,109.190952


The location [0] is correct, so that we could continue with others

In [176]:
# define the method to get nearby venue
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            foursquare_client_id, 
            foursquare_client_secret,  
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [181]:
cluster_centers

[(109.18735854453924, 12.239193561521004),
 (109.18959278651953, 12.233001804495256),
 (109.18977621736579, 12.235607269516706),
 (109.18581834024927, 12.236520375009738),
 (109.19263147760965, 12.240089203781116),
 (109.1872811268575, 12.234373192681614),
 (109.18992867330411, 12.237969172715324),
 (109.19224526089269, 12.233648289771844),
 (109.19280647371826, 12.236189806677595),
 (109.189999747699, 12.240600867063888),
 (109.1882175, 12.2402513)]

In [214]:
df = pd.DataFrame(columns = ['Longitude', 'Latitude', 'Neighborhood'])
for i, point in enumerate(cluster_centers):
    df = df.append({'Longitude': point[0], 'Latitude': point[1], 'Neighborhood': candidate_area_addresses[i]}, ignore_index=True)
df

,Longitude,Latitude,Neighborhood
0,109.187359,12.239194,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra..."
1,109.189593,12.233002,"Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khán..."
2,109.189776,12.235607,"75/1 Nguyễn Thị Minh Khai, Phước Hoà, Thành ph..."
3,109.185818,12.236520,"58 Trần Nhật Duật, Phước Hoà, Thành phố Nha Tr..."
4,109.192631,12.240089,"89A Bạch Đằng, Tân Lập, Thành phố Nha Trang, K..."
5,109.187281,12.234373,"Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khán..."
6,109.189929,12.237969,"160a Mê Linh, Tân Lập, Thành phố Nha Trang, Kh..."
7,109.192245,12.233648,"Biệt Thự, Thành phố Nha Trang, Khánh Hòa 650000"
8,109.192806,12.236190,"34/2/11 Nguyễn Thiện Thuật, Tân Lập, Thành phố..."
9,109.190000,12.240601,"75 Hồng Bàng, Tân Lập, Thành phố Nha Trang, Kh..."


In [215]:
# get nearby venues for all
candidate_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000
Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000
75/1 Nguyễn Thị Minh Khai, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000
58 Trần Nhật Duật, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000
89A Bạch Đằng, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000
Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khánh Hòa
160a Mê Linh, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000
Biệt Thự, Thành phố Nha Trang, Khánh Hòa 650000
34/2/11 Nguyễn Thiện Thuật, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000
75 Hồng Bàng, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000
40 Le Dai Hanh, Nha Trang, Viet Nam


In [216]:
# prepare dataset use most common venues (to avoid noise of small venues)
candidate_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",12.239194,109.187359,An Cafe,12.240180,109.188090,Café
1,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",12.239194,109.187359,Kiwami,12.240153,109.191731,Sushi Restaurant
2,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",12.239194,109.187359,Bún Cá Nguyên Loan,12.241141,109.188283,Asian Restaurant
3,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",12.239194,109.187359,69 Bach Dang,12.240993,109.190783,Asian Restaurant
4,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",12.239194,109.187359,Vietsky Hotel,12.237872,109.190756,Hotel


In [217]:
candidate_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"160a Mê Linh, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000",51,51,51,51,51,51
"34/2/11 Nguyễn Thiện Thuật, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000",100,100,100,100,100,100
"40 Le Dai Hanh, Nha Trang, Viet Nam",27,27,27,27,27,27
"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000",21,21,21,21,21,21
"58 Trần Nhật Duật, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000",9,9,9,9,9,9
"75 Hồng Bàng, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000",18,18,18,18,18,18
"75/1 Nguyễn Thị Minh Khai, Phước Hoà, Thành phố Nha Trang, Khánh Hòa 650000",30,30,30,30,30,30
"89A Bạch Đằng, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000",26,26,26,26,26,26
"Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khánh Hòa",6,6,6,6,6,6


In [218]:
print('There are {} uniques categories.'.format(len(candidate_venues['Venue Category'].unique())))

There are 51 uniques categories.


In [219]:
# one hot encoding
candidate_onehot = pd.get_dummies(candidate_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
candidate_onehot['Neighborhood'] = candidate_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [candidate_onehot.columns[-1]] + list(candidate_onehot.columns[:-1])
candidate_onehot = candidate_onehot[fixed_columns]

candidate_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Breakfast Spot,Café,...,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Tourist Information Center,Vietnamese Restaurant
0,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [220]:
candidate_grouped = candidate_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 5

for hood in candidate_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = candidate_grouped[candidate_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----160a Mê Linh, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000----
                   venue  freq
0                   Café  0.18
1                  Hotel  0.14
2  Vietnamese Restaurant  0.12
3       Sushi Restaurant  0.08
4            Coffee Shop  0.08


----34/2/11 Nguyễn Thiện Thuật, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000----
                   venue  freq
0                  Hotel  0.15
1  Vietnamese Restaurant  0.12
2                   Café  0.09
3            Coffee Shop  0.08
4     Italian Restaurant  0.05


----40 Le Dai Hanh, Nha Trang, Viet Nam----
                   venue  freq
0                   Café  0.30
1  Vietnamese Restaurant  0.11
2                  Hotel  0.11
3       Asian Restaurant  0.07
4     Seafood Restaurant  0.07


----56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Trang, Khánh Hòa 650000----
                   venue  freq
0                   Café  0.24
1                  Hotel  0.14
2  Vietnamese Restaurant  0.10
3       Asian Restaurant  0.10
4     Seaf

In [221]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = candidate_grouped['Neighborhood']

for ind in np.arange(candidate_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(candidate_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"160a Mê Linh, Tân Lập, Thành phố Nha Trang, Kh...",Café,Hotel,Vietnamese Restaurant,Sushi Restaurant,Coffee Shop
1,"34/2/11 Nguyễn Thiện Thuật, Tân Lập, Thành phố...",Hotel,Vietnamese Restaurant,Café,Coffee Shop,Italian Restaurant
2,"40 Le Dai Hanh, Nha Trang, Viet Nam",Café,Vietnamese Restaurant,Hotel,Seafood Restaurant,Asian Restaurant
3,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",Café,Hotel,Vietnamese Restaurant,Seafood Restaurant,Asian Restaurant
4,"58 Trần Nhật Duật, Phước Hoà, Thành phố Nha Tr...",Café,Vietnamese Restaurant,Tourist Information Center,Gym,Soccer Field


In [222]:
# clustering 
# set number of clusters
kclusters = 3

grouped_clustering = candidate_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 2, 2, 0, 2, 1, 1, 0, 2, 1], dtype=int32)

In [223]:
grouped_clustering.head()

,American Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Breakfast Spot,Café,Coffee Shop,...,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Tourist Information Center,Vietnamese Restaurant
0,0.019608,0.00,0.039216,0.019608,0.019608,0.00,0.019608,0.00,0.176471,0.078431,...,0.019608,0.00,0.00,0.00,0.00,0.078431,0.00,0.000000,0.000000,0.117647
1,0.010000,0.01,0.000000,0.010000,0.000000,0.03,0.010000,0.01,0.090000,0.080000,...,0.040000,0.01,0.01,0.01,0.01,0.040000,0.01,0.000000,0.010000,0.120000
2,0.000000,0.00,0.074074,0.037037,0.000000,0.00,0.000000,0.00,0.296296,0.074074,...,0.000000,0.00,0.00,0.00,0.00,0.037037,0.00,0.037037,0.000000,0.111111
3,0.000000,0.00,0.095238,0.047619,0.000000,0.00,0.000000,0.00,0.238095,0.047619,...,0.000000,0.00,0.00,0.00,0.00,0.047619,0.00,0.000000,0.000000,0.095238
4,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.444444,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.111111,0.111111


In [244]:
grouped_clustering.columns

Index(['American Restaurant', 'Art Gallery', 'Asian Restaurant', 'BBQ Joint',
       'Bakery', 'Bar', 'Bistro', 'Breakfast Spot', 'Café', 'Coffee Shop',
       'Comfort Food Restaurant', 'Convenience Store', 'Dessert Shop',
       'Distillery', 'French Restaurant', 'Greek Restaurant', 'Gym',
       'Hookah Bar', 'Hostel', 'Hotel', 'Ice Cream Shop', 'Indian Restaurant',
       'Italian Restaurant', 'Japanese Restaurant', 'Jewelry Store',
       'Juice Bar', 'Lounge', 'Market', 'Modern European Restaurant', 'Motel',
       'Pizza Place', 'Plaza', 'Pop-Up Shop', 'Pub', 'Resort',
       'Russian Restaurant', 'Seafood Restaurant', 'Smoothie Shop',
       'Snack Place', 'Soccer Field', 'Soup Place', 'Spa',
       'Spanish Restaurant', 'Sporting Goods Shop', 'Sports Bar', 'Steakhouse',
       'Sushi Restaurant', 'Taco Place', 'Thai Restaurant',
       'Tourist Information Center', 'Vietnamese Restaurant'],
      dtype='object')

In [224]:
# merge all candidates with the label
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [227]:
candidate_merged = df.copy()

# merge candidate_grouped with df to add latitude/longitude for each neighborhood
candidate_merged = candidate_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

candidate_merged.head() # check the last columns!

,Longitude,Latitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,109.187359,12.239194,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",2,Café,Hotel,Vietnamese Restaurant,Seafood Restaurant,Asian Restaurant
1,109.189593,12.233002,"Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khán...",2,Hotel,Café,Vietnamese Restaurant,Seafood Restaurant,Hostel
2,109.189776,12.235607,"75/1 Nguyễn Thị Minh Khai, Phước Hoà, Thành ph...",1,Hotel,Vietnamese Restaurant,Café,Coffee Shop,Seafood Restaurant
3,109.185818,12.236520,"58 Trần Nhật Duật, Phước Hoà, Thành phố Nha Tr...",0,Café,Vietnamese Restaurant,Tourist Information Center,Gym,Soccer Field
4,109.192631,12.240089,"89A Bạch Đằng, Tân Lập, Thành phố Nha Trang, K...",1,Café,Hotel,Coffee Shop,Spa,Asian Restaurant


In [236]:
# check label of success coffee shop
candidate_merged[candidate_merged['Neighborhood'] == success_coffee_shop_address]

,Longitude,Latitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,109.188217,12.240251,"40 Le Dai Hanh, Nha Trang, Viet Nam",2,Café,Vietnamese Restaurant,Hotel,Seafood Restaurant,Asian Restaurant


In [237]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=nhatrang_center, zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(candidate_merged['Latitude'], candidate_merged['Longitude'], candidate_merged['Neighborhood'], candidate_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    color = 'red' if cluster == 2 else 'blue'
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# add Marker to success coffee shop
folium.Marker([success_coffee_shop_coordinates[0], success_coffee_shop_coordinates[1]], popup=addr).add_to(map_clusters) 
    
map_clusters

In [243]:
# summary information
best_candidates = candidate_merged[candidate_merged['Cluster Labels'] == 2]
print('Number of locations that same cluster with success coffee shop: ', best_candidates['Neighborhood'].size)
best_candidates

Number of locations that same cluster with success coffee shop:  4


,Longitude,Latitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,109.187359,12.239194,"56 Nguyễn Hữu Huân, Tân Lập, Thành phố Nha Tra...",2,Café,Hotel,Vietnamese Restaurant,Seafood Restaurant,Asian Restaurant
1,109.189593,12.233002,"Biệt Thự, Phước Hoà, Thành phố Nha Trang, Khán...",2,Hotel,Café,Vietnamese Restaurant,Seafood Restaurant,Hostel
9,109.190000,12.240601,"75 Hồng Bàng, Tân Lập, Thành phố Nha Trang, Kh...",2,Café,Hotel,Asian Restaurant,Spa,Convenience Store
10,109.188217,12.240251,"40 Le Dai Hanh, Nha Trang, Viet Nam",2,Café,Vietnamese Restaurant,Hotel,Seafood Restaurant,Asian Restaurant


## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of coffee shops in Nha Trang (~400 in our initial area of interest which was around Thap Tram Huong), there are pockets of low coffee shop density fairly close to city center. Highest concentration of coffee shops was detected north and south from Thap Tram Huong, so we focused our attention to areas south-west and north-west. But north-west locations are expensive so that we focused on south-west that nearby the boroughs between Tan Lap and Phuoc Hoa. 

After directing our attention to this more narrow area of interest (covering approx. 5x5km south-west from Thap Tram Huong) we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two coffee shops in radius of 500m and those with an coffee shop closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 10 zones containing largest number of potential new coffee shop locations based on number of and distance to existing venues.

Then, we add the successful coffee shop to the candidates and make deeper analysis. Because the stakeholder had a successful coffee shop, we cluster this coffee shop as well as candidates. The best candidates are the one that are grouped within same cluster with the success coffee shop with response to the features that extracted from 5 most common venues. Finally, we got 4 best candidates including the success coffee shop, so that we could extract 3 best candidates.

This, of course, does not imply that those zones are actually optimal locations for a new coffee shop! Purpose of this analysis was to only provide info on areas close to Nha Trang center but not crowded with existing coffee shops - it is entirely possible that there is a very good reason for small number of coffee shops in any of those areas, reasons which would make them unsuitable for a new coffee shop regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Nha Trang areas close to center with low number of coffee shops in order to aid stakeholders in narrowing down the search for optimal location for a new coffee shop. By calculating coffee shop density distribution from Foursquare data we have first identified general boroughs that justify further analysis (Tan Lap and Phuoc Hoa), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby coffee shops. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal coffee shop location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.